In [9]:
from pprint import pprint
from functools import reduce
import brotli
import pickle


with open("./dump/dbeaver_dbeaver/commits.szhat".replace("\\","/"), "rb") as handler:
    # print(handler.read())
    data = handler.read()
    decompressed_data = brotli.decompress(data)
    obj = pickle.loads(decompressed_data)

# print(c[1].files)
# avg = reduce(lambda x, y: x+y, map(lambda c: c.files, commits))/ len(commits)
# c = commits[0]

# print(c.stats.additions)

In [10]:
print([o for o in obj if o.files is None])

[Commit(sha="fffdc7e483462617869d63f38d4f6473b084c876")]


## Gathering the data

In [62]:
import glob
import pickle
from utils.metrics.contributors import Contributors
from collections import defaultdict
from pprint import pprint
import pandas as pd

repo_dicts = defaultdict(lambda: {})
metric_dicts = defaultdict(lambda: defaultdict(lambda: {}))
for p in glob.glob(".\intermediate\*\*.pickle"):
    with open(p, "rb") as file:
        metric = pickle.load(file)
        dic = metric.__dict__
        repo_name = dic.pop("repo_name")
        metric_type = p.split('\\')[-1].split(".")[0]
        # for metric_name, val in dic.items():
            # (metric_dicts[metric_type])[metric_name] += [val]
        metric_dicts[metric_type][repo_name] |= dic
        repo_dicts[repo_name] |= dic

data = pd.DataFrame.from_dict(repo_dicts)
print(f"Metrics count: {len(data)}")
print(f"Repo    count: {len(data.keys())}")
display(data)
data.to_csv("metrics.csv",index=True)

Metrics count: 101
Repo    count: 143


,Activiti/Activiti,airbnb/lottie-android,alibaba/arthas,alibaba/fastjson,alibaba/Sentinel,alibaba/spring-cloud-alibaba,android-async-http/android-async-http,ansible/ansible,apache/beam,apache/cassandra,...,swagger-api/swagger-core,tachiyomiorg/tachiyomi,tdebatty/java-LSH,telegramdesktop/tdesktop,Tencent/tinker,testcontainers/testcontainers-java,TheCherno/Hazel,tony19/logback-android,vifm/vifm,zxing/zxing
[Commits] Count,1.076800e+04,1358.000000,1659.000000,3933.000000,713.000000,1838.0,899.000000,NaN,3.353400e+04,27038.0,...,3.985000e+03,3937.000000,82.0,11077.0,735.0,2947.0,243.0,4238.0,11773.0,3620.000000
[Commits] Days since first,4.165000e+03,1917.000000,1169.000000,3757.000000,1208.000000,1441.0,3918.000000,NaN,2.525000e+03,4638.0,...,3.759000e+03,2241.000000,2480.0,2755.0,1878.0,2406.0,1123.0,5660.0,4681.0,5134.000000
[Commits] Days since last,1.000000e+01,6.000000,8.000000,31.000000,58.000000,11.0,298.000000,NaN,0.000000e+00,0.0,...,3.900000e+01,9.000000,871.0,3.0,12.0,10.0,59.0,541.0,3.0,21.000000
[Commits] Total lines added,8.310435e+06,122432.000000,244391.000000,799527.000000,167770.000000,553069.0,44931.000000,NaN,1.415426e+07,3239411.0,...,1.190023e+06,342210.000000,5969.0,2418054.0,110864.0,102479.0,50664.0,493487.0,1071378.0,831281.000000
[Commits] Total lines deleted,5.064341e+06,59739.000000,40864.000000,191777.000000,25075.000000,303082.0,19781.000000,NaN,7.319061e+06,1712123.0,...,1.025639e+06,227873.000000,3228.0,1620051.0,32416.0,38560.0,13563.0,386642.0,505776.0,617442.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Workflow Runs] Average failure duration (ms),1.582054e+05,51468.809074,146544.811321,907521.126761,52263.888889,NaN,0.000000,NaN,3.737057e+05,NaN,...,1.045638e+04,3173.298178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
[Workflow Runs] Average successes per day,2.371795e+00,3.148810,4.790960,2.535714,1.469388,NaN,1.818182,NaN,8.473559e+01,NaN,...,4.715190e+00,28.871972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.707071
[Workflow Runs] Average successes per day (True),5.525687e-02,0.283951,0.724167,0.019383,0.054587,NaN,0.005103,NaN,2.019043e+01,NaN,...,1.969337e-01,7.486765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045862
[Workflow Runs] Average fails per day,2.258065e+00,2.108696,3.250000,2.419355,1.583333,NaN,NaN,NaN,1.052123e+01,NaN,...,1.903226e+00,1.620253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calculation correlation

In [182]:
from itertools import combinations_with_replacement


values = {name:data.loc[name] for name in data.index}
correlations = pd.DataFrame(values).corr(method='pearson').values
frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)
# frame = frame[frame[frame.index] > 0.8]
frame.to_csv("correlations.csv", index=True)
display(frame)

for g1, g2 in combinations_with_replacement(metric_dicts.items(), 2):
    _, inner_dict1 = g1
    _, inner_dict2 = g2

    res = {name:inner_dict1[name] | inner_dict2[name] for name in inner_dict1}
    data = pd.DataFrame(res).transpose().corr(method='pearson')
    # data = data[data[data.columns] > 0.7]
    metric1_names = list(list(inner_dict1.values())[0].keys())
    metric2_names = list(list(inner_dict2.values())[0].keys())

    df = data[metric1_names].loc[metric2_names]
    display(df)

C:\Users\79134\AppData\Local\Temp/ipykernel_6732/2744595380.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)


,[Workflow Runs] Count,[Workflow Runs] Successful to total,[Workflow Runs] Failed to total,[Workflow Runs] Skipped to total,[Workflow Runs] Average duration (ms),[Workflow Runs] Average success duration (ms),[Workflow Runs] Average failure duration (ms),[Workflow Runs] Average successes per day,[Workflow Runs] Average successes per day (True),[Workflow Runs] Average fails per day,[Workflow Runs] Average fails per day (True)
[Workflow Runs] Count,1.0,0.219522,-0.798529,0.375252,-0.632749,-0.604537,-0.601812,0.952003,0.953062,0.021508,0.769224
[Workflow Runs] Successful to total,0.0,1.000000,-0.166466,-0.070262,-0.855037,-0.810219,-0.859848,0.066952,0.151518,-0.786257,-0.135335
[Workflow Runs] Failed to total,0.0,0.000000,1.000000,-0.405811,0.515445,0.414779,0.610837,-0.773567,-0.776088,0.091726,-0.505105
[Workflow Runs] Skipped to total,0.0,0.000000,0.000000,1.000000,-0.165047,-0.139718,-0.164242,0.170276,0.117850,-0.145305,-0.071473
[Workflow Runs] Average duration (ms),0.0,0.000000,0.000000,0.000000,1.000000,0.975158,0.937965,-0.483150,-0.555561,0.590633,-0.290534
[Workflow Runs] Average success duration (ms),0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.844159,-0.457844,-0.530849,0.537212,-0.319357
[Workflow Runs] Average failure duration (ms),0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-0.461611,-0.531676,0.629382,-0.210367
[Workflow Runs] Average successes per day,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.973593,0.254786,0.864026
[Workflow Runs] Average successes per day (True),0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.110172,0.877424
[Workflow Runs] Average fails per day,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.436948


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Commits] Count,1.000000,0.335567,-0.192595,0.265015,0.247250,0.015368,0.063452,0.006517,0.193460,0.787241,0.518628,0.172862
[Commits] Days since first,0.335567,1.000000,-0.041945,0.045129,0.029374,0.029289,0.029917,0.047151,0.138211,-0.019814,0.255524,-0.103033
[Commits] Days since last,-0.192595,-0.041945,1.000000,-0.088794,-0.070471,-0.057379,-0.060442,-0.058999,-0.213858,-0.043646,-0.078474,-0.051571
[Commits] Total lines added,0.265015,0.045129,-0.088794,1.000000,0.957784,0.728515,0.876013,0.601610,-0.047535,0.333852,0.117571,0.411373
[Commits] Total lines deleted,0.247250,0.029374,-0.070471,0.957784,1.000000,0.518655,0.728279,0.367479,-0.057427,0.347306,0.103941,0.474824
[Commits] Average additions,0.015368,0.029289,-0.057379,0.728515,0.518655,1.000000,0.956418,0.972880,-0.061333,0.013500,-0.004258,0.115140
[Commits] Average deletions,0.063452,0.029917,-0.060442,0.876013,0.728279,0.956418,1.000000,0.891323,-0.088905,0.089729,0.009023,0.242067
[Commits] Average files changed,0.006517,0.047151,-0.058999,0.601610,0.367479,0.972880,0.891323,1.000000,-0.061593,-0.030770,0.001992,0.030485
[Commits] Average message length (chars),0.193460,0.138211,-0.213858,-0.047535,-0.057427,-0.061333,-0.088905,-0.061593,1.000000,0.077814,0.093533,0.123142
[Commits] Per day,0.787241,-0.019814,-0.043646,0.333852,0.347306,0.013500,0.089729,-0.030770,0.077814,1.000000,0.257906,0.378504


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Contributors] Count,0.586377,0.185197,-0.364537,0.096581,0.061138,0.019487,0.011836,0.048337,0.308755,0.423105,0.212184,-0.003429
[Contributors Top-100] Average commits,0.882333,0.419183,-0.129366,0.204415,0.205878,-0.025477,0.020503,-0.048202,0.223831,0.687642,0.508456,0.177410
[Contributors Top-100] Average participation weeks,0.879488,0.299433,-0.185787,0.134204,0.111827,-0.049052,-0.034564,-0.049039,0.305011,0.684087,0.621361,0.041229
[Contributors Top-100] Average additions,0.320961,0.011435,-0.079485,0.884985,0.961216,0.362991,0.586390,0.196779,0.009883,0.455474,0.167101,0.952285
[Contributors Top-100] Average deletions,0.324046,0.020512,-0.073263,0.886824,0.966581,0.359125,0.586828,0.198052,0.002109,0.460482,0.175741,0.954807


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Forks] Count,0.226399,0.003952,-0.126322,-0.008338,-0.011380,-0.015319,-0.017949,-0.001545,0.066328,0.183341,-0.007763,-0.028482
[Forks] Per day (True),0.172706,-0.123414,-0.139745,-0.018781,-0.019955,-0.019991,-0.025105,-0.006097,0.017591,0.156964,0.011635,-0.028078
[Forks] Max per day,0.047462,0.122373,-0.052331,-0.040716,-0.034117,-0.045794,-0.046589,-0.032164,-0.028719,0.011227,-0.018104,-0.037157


...

,[Releases] Count,[Releases] Tags,[Releases] Total downloads,[Releases] Average body len (chars),[Releases] Average title len (chars),[Releases] Average assets,[Releases] Average asset downloads,[Releases] Average asset size,[Releases] Drafts to total,[Releases] Prereleases to total,[Releases] Per day created (True),[Releases] Per day published (True),[Releases] Downloads per day (True)
[Stars] Count,0.077738,-0.032474,0.168532,0.339122,-0.044042,0.024560,0.204032,-0.027660,NaN,-0.058159,0.264582,0.315394,0.299988
[Stars] Per day,-0.049481,0.122662,-0.067021,0.149216,0.419433,-0.092070,0.022515,-0.038819,NaN,0.119231,-0.155727,-0.173984,-0.079782
[Stars] Per day (True),0.162191,-0.060299,0.249284,0.196043,-0.072214,0.047151,0.317597,-0.002123,NaN,-0.122431,0.301684,0.362492,0.484213
[Stars] Maximum per day,-0.011212,0.203998,-0.109482,0.052068,0.192189,-0.124790,-0.079070,-0.024898,NaN,0.150191,-0.124217,-0.152644,-0.114987


,[Releases] Count,[Releases] Tags,[Releases] Total downloads,[Releases] Average body len (chars),[Releases] Average title len (chars),[Releases] Average assets,[Releases] Average asset downloads,[Releases] Average asset size,[Releases] Drafts to total,[Releases] Prereleases to total,[Releases] Per day created (True),[Releases] Per day published (True),[Releases] Downloads per day (True)
[Workflow Runs] Count,0.068529,0.211874,0.037157,0.211518,-0.020121,-0.032765,0.077715,-0.021261,NaN,-0.079790,0.085690,0.071940,0.042963
[Workflow Runs] Successful to total,0.130004,0.099656,-0.089882,0.105643,0.100604,-0.005473,0.145477,0.091758,NaN,0.040651,0.005190,0.009171,-0.069632
[Workflow Runs] Failed to total,-0.031071,0.049702,-0.057363,0.060017,-0.006361,-0.141538,0.017509,-0.008865,NaN,0.130542,-0.018913,-0.026984,-0.049440
[Workflow Runs] Skipped to total,0.193046,-0.001229,0.001246,-0.034672,-0.139598,-0.038077,-0.003175,-0.013578,NaN,-0.033283,0.155270,0.114267,-0.004862
[Workflow Runs] Average duration (ms),0.126659,0.059890,-0.100936,0.027380,-0.146871,-0.110773,-0.077378,-0.000545,NaN,0.027759,0.032417,0.014095,-0.099797
[Workflow Runs] Average success duration (ms),0.093301,0.042995,-0.078098,0.062335,-0.118171,-0.084164,-0.066041,0.047738,NaN,0.087342,-0.001731,-0.015893,-0.077760
[Workflow Runs] Average failure duration (ms),0.120003,0.079677,-0.094191,0.010424,-0.072038,-0.074668,-0.087816,-0.046021,NaN,0.015710,0.128995,0.106829,-0.087528
[Workflow Runs] Average successes per day,0.009809,0.412120,0.111945,0.270728,0.050910,-0.045178,0.048993,-0.035585,NaN,0.035163,0.043006,0.040625,0.098354
[Workflow Runs] Average successes per day (True),0.004904,0.122570,0.169333,0.251328,0.041717,-0.005278,0.093174,-0.024751,NaN,-0.078752,0.100604,0.106211,0.159159
[Workflow Runs] Average fails per day,0.077209,0.531480,-0.119169,0.213711,0.076537,-0.084245,-0.127233,-0.079380,NaN,0.140899,-0.030028,-0.046918,-0.104272


,[Stars] Count,[Stars] Per day,[Stars] Per day (True),[Stars] Maximum per day
[Stars] Count,1.000000,-0.097997,0.635056,0.037791
[Stars] Per day,-0.097997,1.000000,-0.160686,0.456357
[Stars] Per day (True),0.635056,-0.160686,1.000000,-0.035975
[Stars] Maximum per day,0.037791,0.456357,-0.035975,1.000000


,[Stars] Count,[Stars] Per day,[Stars] Per day (True),[Stars] Maximum per day
[Workflow Runs] Count,-0.008147,-0.057613,-0.008290,-0.024747
[Workflow Runs] Successful to total,0.152596,0.092546,0.111310,-0.016269
[Workflow Runs] Failed to total,0.035346,-0.113451,-0.002313,-0.055315
[Workflow Runs] Skipped to total,-0.028803,-0.047737,-0.035471,-0.027035
[Workflow Runs] Average duration (ms),-0.057788,-0.059952,-0.088957,0.080126
[Workflow Runs] Average success duration (ms),-0.061356,-0.083223,-0.083966,-0.017593
[Workflow Runs] Average failure duration (ms),-0.024601,-0.077758,-0.047141,-0.018737
[Workflow Runs] Average successes per day,-0.042377,-0.045738,-0.038339,0.023152
[Workflow Runs] Average successes per day (True),-0.061717,-0.106641,-0.020316,-0.095864
[Workflow Runs] Average fails per day,-0.017851,0.080549,-0.061643,0.249771


,[Workflow Runs] Count,[Workflow Runs] Successful to total,[Workflow Runs] Failed to total,[Workflow Runs] Skipped to total,[Workflow Runs] Average duration (ms),[Workflow Runs] Average success duration (ms),[Workflow Runs] Average failure duration (ms),[Workflow Runs] Average successes per day,[Workflow Runs] Average successes per day (True),[Workflow Runs] Average fails per day,[Workflow Runs] Average fails per day (True)
[Workflow Runs] Count,1.000000,0.152580,-0.034575,0.491749,-0.026756,-0.018538,-0.030801,0.902264,0.866121,0.325432,0.624084
[Workflow Runs] Successful to total,0.152580,1.000000,0.161724,-0.044245,-0.459370,-0.368742,-0.504941,0.034527,0.071305,-0.312030,-0.126420
[Workflow Runs] Failed to total,-0.034575,0.161724,1.000000,-0.016079,0.386987,0.252864,0.504417,-0.154388,-0.178060,0.269914,0.083592
[Workflow Runs] Skipped to total,0.491749,-0.044245,-0.016079,1.000000,0.094759,0.098314,0.098170,0.197024,0.121671,0.107834,0.027471
[Workflow Runs] Average duration (ms),-0.026756,-0.459370,0.386987,0.094759,1.000000,0.880212,0.766048,0.007959,-0.049915,0.504814,0.082977
[Workflow Runs] Average success duration (ms),-0.018538,-0.368742,0.252864,0.098314,0.880212,1.000000,0.527182,0.031456,-0.035159,0.423089,0.028866
[Workflow Runs] Average failure duration (ms),-0.030801,-0.504941,0.504417,0.098170,0.766048,0.527182,1.000000,0.008564,-0.048501,0.505252,0.143887
[Workflow Runs] Average successes per day,0.902264,0.034527,-0.154388,0.197024,0.007959,0.031456,0.008564,1.000000,0.868637,0.509674,0.652278
[Workflow Runs] Average successes per day (True),0.866121,0.071305,-0.178060,0.121671,-0.049915,-0.035159,-0.048501,0.868637,1.000000,0.268562,0.744488
[Workflow Runs] Average fails per day,0.325432,-0.312030,0.269914,0.107834,0.504814,0.423089,0.505252,0.509674,0.268562,1.000000,0.497526
